In [28]:
library(tidyverse)
library(dplyr)
library(glmnet)
library(car)
library(broom)

In [29]:
data <- read_csv("house_data.csv") %>% drop_na()

New names:
* `` -> ...1

Rows: 20640 Columns: 11
-- Column specification --------------------------------------------------------
Delimiter: ","
chr  (1): ocean_proximity
dbl (10): ...1, longitude, latitude, housing_median_age, total_rooms, total_...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [30]:
# preprocessing from the proposal

data <- data %>% mutate(rooms_per_household = total_rooms/households,
                       bedrooms_per_household = total_bedrooms/households,
                       population_per_household = population/households) %>%
                select(-...1) 
data <- data %>% relocate(median_house_value, .after = last_col())

head(data)

longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,rooms_per_household,bedrooms_per_household,population_per_household,median_house_value
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
-122.23,37.88,41,880,129,322,126,8.3252,NEAR BAY,6.984127,1.0238095,2.555556,452600
-122.22,37.86,21,7099,1106,2401,1138,8.3014,NEAR BAY,6.238137,0.9718805,2.109842,358500
-122.24,37.85,52,1467,190,496,177,7.2574,NEAR BAY,8.288136,1.0734463,2.802260,352100
-122.25,37.85,52,1274,235,558,219,5.6431,NEAR BAY,5.817352,1.0730594,2.547945,341300
-122.25,37.85,52,1627,280,565,259,3.8462,NEAR BAY,6.281853,1.0810811,2.181467,342200
-122.25,37.85,52,919,213,413,193,4.0368,NEAR BAY,4.761658,1.1036269,2.139896,269700


### Fitting a full linear model

In [35]:
full_model <- lm(median_house_value~., data = data)
tidy(full_model)
vif(full_model)

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-2.355682e+06,8.861237e+04,-26.584116,4.159506e-153
longitude,-2.766339e+04,1.026361e+03,-26.952876,2.962510e-157
latitude,-2.635291e+04,1.014102e+03,-25.986448,1.690167e-146
housing_median_age,1.073456e+03,4.379668e+01,24.509976,8.946624e-131
total_rooms,-3.589039e+00,9.357749e-01,-3.835366,1.257542e-04
total_bedrooms,5.946919e+01,7.890935e+00,7.536394,5.032696e-14
population,-3.887653e+01,1.107548e+00,-35.101428,3.817785e-262
households,8.275246e+01,8.310903e+00,9.957096,2.653919e-23
median_income,4.033897e+04,4.128793e+02,97.701587,0.000000e+00


,GVIF,Df,GVIF^(1/(2*Df))
longitude,18.427132,1,4.292684
latitude,20.452759,1,4.522473
housing_median_age,1.325266,1,1.151202
total_rooms,18.222064,1,4.268731
total_bedrooms,48.179240,1,6.941127
population,6.864198,1,2.619961
households,43.989398,1,6.632450
median_income,2.679626,1,1.636956
ocean_proximity,4.173184,4,1.195524
rooms_per_household,12.356066,1,3.515120


### Using LASSO to select a generative model

Other than using forward/backward selection to create a model, we would like to see if LASSO generates a model with different parameters. Simutaneously, we would like to explore whether LASSO yields a better or worse performing model. 

We will choose our LASSO with a $\lambda$ value that gives the smallest $MSE_{test}$.

To avoid bias, we will first split our data into two evenly divided portions -- one for variable selection, and the other for fitting the inference model. We will name the two portions `data_selection` and `data_inference`.

In [11]:
set.seed(1234)
indices <- sample.int(nrow(data), nrow(data)/2)
data_selection <- data[indices, ]
data_inference <- data[-indices, ]

In the following cell, we create the X and Y portion of the variable selection portion. Because we have a categorical variable, `ocean_promixity`, we need to create group dummy variables in order to fit a LASSO model. The code to do so is referenced [here](https://stackoverflow.com/questions/46865838/using-lasso-in-r-with-categorical-variables). 

In [12]:
set.seed(1234)

data_selection <- data_selection %>% mutate(ocean_proximity = as.factor(ocean_proximity))
data_selection_Y <- data_selection %>% select(median_house_value) %>% as.matrix()

vars_name <- data_selection %>% 
  select(-median_house_value) %>% 
  colnames() %>% 
  str_c(collapse = "+") 

model_string <- paste("median_house_value  ~",vars_name )

data_selection_X <- model.matrix(as.formula(model_string), data_selection)


Then, we train the regression using LASSO and find the optimal value of the hyperparameter $\lambda$.

In [13]:
set.seed(1234)

# referenced from worksheet 11
housing_lambda_LASSO <- cv.glmnet(
  x = data_selection_X, y = data_selection_Y,
  alpha = 1,
  lambda = 10^seq(3, -2, by = -.1)
)

housing_lambda_min_MSE_LASSO <- round(housing_lambda_LASSO$lambda.min, 4)
cat("The best lambda value is", housing_lambda_min_MSE_LASSO)

The best lambda value is 199.5262

Now, we train another LASSO model using the optimal lambda we found.

In [14]:
housing_LASSO_min <- glmnet(
  x = data_selection_X, y = data_selection_Y,
  alpha = 1,
  lambda = housing_lambda_min_MSE_LASSO
)

In [18]:
housing_LASSO_min$beta

16 x 1 sparse Matrix of class "dgCMatrix"
                                     s0
(Intercept)                .           
longitude                 -2.277566e+04
latitude                  -2.154839e+04
housing_median_age         1.086667e+03
total_rooms               -6.229462e-02
total_bedrooms             4.519371e+01
population                -3.411527e+01
households                 6.847233e+01
median_income              3.960898e+04
ocean_proximityINLAND     -4.259871e+04
ocean_proximityISLAND      1.585719e+05
ocean_proximityNEAR BAY    2.162682e+03
ocean_proximityNEAR OCEAN  6.851486e+03
rooms_per_household       -4.798228e+03
bedrooms_per_household     2.445515e+04
population_per_household   3.871722e+02